In [4]:
import smtplib

In [5]:
from email.mime.text import MIMEText

In [9]:
# 시스템 라이브러리 활용 패키지
import os

# 이메일 메시지에 형식을 활용하기 위한 패키지
from email.mime.multipart import MIMEMultipart

# 이메일 메시지 프로토콜 처리기(엔코더)
from email import encoders

# 텍스트 형식 메일
from email.mime.text import MIMEText

# 이미지 형식 메일
from email.mime.image import MIMEImage

# 오디오 형식 메일
from email.mime.audio import MIMEAudio

# 이 모든 정보들을 관장하는 객체
from email.mime.base import MIMEBase

In [15]:
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info["smtp_server"], smtp_info["smtp_port"]) as server:
        # TLS 보안 설정 및 시작 (보안 정책상 필수)
        server.starttls()
        
        # 로그인
        server.login(smtp_info["smtp_user_id"], smtp_info["smtp_user_pw"])
        
        # 이메일 전송
        # 메시지 전송시 반드시 as_string()으로 변환하여 전송해야함
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공!!!')
        else:
            print(res)

In [17]:
smtp_info = dict({
    "smtp_server": "smtp.naver.com",
    "smtp_user_id": "사용자계정@naver.com",
    "smtp_user_pw": "비밀번호",
    "smtp_port": 587
})

title = "가즈아!!!!!!"
content = "날아오르라 주식이여!!!!"
sender = smtp_info['smtp_user_id']
receiver = smtp_info['smtp_user_id']

msg = MIMEText(_text = content, _charset = "utf-8")

msg['Subject'] = title
msg['From'] = sender
msg['To'] = receiver

send_email(smtp_info, msg)

이메일 전송 성공!!!
